In [1]:
import pandas as pd
import numpy as np

import requests
import openpyxl as xl
import time
import json

import folium
import re

In [6]:
import warnings
warnings.filterwarnings(action='ignore') # 경고메세지 숨기기

In [ ]:
#import re
#center_info['자치구'] = center_info['시 설 명'].map(lambda x: re.findall('[가-힣]+',x)[0] + '구')
#center_info['자치구'] = center_info['자치구'].map(lambda x: x.replace('중구구', '중구'))

In [5]:
busstop = pd.read_csv("서울시버스정류소좌표데이터.csv", encoding="cp949")
busstop = busstop[["정류소명", "좌표X", "좌표Y"]]
busstop = busstop.rename(columns={"좌표X":"x","좌표Y":"y"})
busstop

,정류소명,x,y
0,종로2가사거리,126.987786,37.569764
1,창경궁.서울대학교병원,126.996520,37.579179
2,명륜3가.성대입구,126.998290,37.582709
3,종로2가.삼일교,126.987507,37.568582
4,혜화동로터리.여운형활동터,127.001694,37.586230
...,...,...,...
11162,우성아파트,127.139326,37.550442
11163,우성아파트,127.140052,37.550620
11164,조일약국,127.123477,37.533686
11165,성내시장,127.125410,37.536199


In [9]:
busstop["행정동"] = ""
 
for i in range(11167):
    x_val = busstop.x[i]
    y_val = busstop.y[i]
    base_url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x='+str(x_val)+'&y='+str(y_val)+'&input_coord=WGS84'
    
    your_api_key = 'd89c853ebcefdd731422cbd0cb4cb9a4'
    headers = {'Authorization':'KakaoAK '+your_api_key}

    api_req = requests.get(base_url,headers=headers)
    busstop["행정동"][i] = json.loads(api_req.text)["documents"][1]['region_3depth_name']

In [11]:
df = busstop.copy()

In [12]:
df

,정류소명,x,y,행정동
0,종로2가사거리,126.987786,37.569764,종로1.2.3.4가동
1,창경궁.서울대학교병원,126.996520,37.579179,종로1.2.3.4가동
2,명륜3가.성대입구,126.998290,37.582709,혜화동
3,종로2가.삼일교,126.987507,37.568582,종로1.2.3.4가동
4,혜화동로터리.여운형활동터,127.001694,37.586230,혜화동
...,...,...,...,...
11162,우성아파트,127.139326,37.550442,천호1동
11163,우성아파트,127.140052,37.550620,암사1동
11164,조일약국,127.123477,37.533686,성내2동
11165,성내시장,127.125410,37.536199,성내2동


In [22]:
sinsa = df[df["행정동"]=="신사동"]
sinsa = sinsa.reset_index()

In [23]:
sinsa["구"] = ""
 
for i in range(34):
    x_val = sinsa.x[i]
    y_val = sinsa.y[i]
    base_url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x='+str(x_val)+'&y='+str(y_val)+'&input_coord=WGS84'
    
    your_api_key = 'd89c853ebcefdd731422cbd0cb4cb9a4'
    headers = {'Authorization':'KakaoAK '+your_api_key}

    api_req = requests.get(base_url,headers=headers)
    sinsa["구"][i] = json.loads(api_req.text)["documents"][1]['region_2depth_name']

In [25]:
ss = sinsa.groupby ("구").count()
ss

,index,정류소명,x,y,행정동
구,,,,,
강남구,16,16,16,16,16
관악구,18,18,18,18,18


In [13]:
# df.to_excel("서울시_버스정류소_좌표데이터.xlsx")

In [14]:
df.isnull().sum()

정류소명    0
x       0
y       0
행정동     0
dtype: int64

In [16]:
bus = df.groupby ("행정동").count()
bus = bus[["정류소명"]]
bus = bus.rename(columns={"정류소명":"count_busstop"})
bus = bus.reset_index()
bus

,행정동,count_busstop
0,가락1동,10
1,가락2동,22
2,가락본동,25
3,가리봉동,10
4,가산동,105
...,...,...
430,효창동,14
431,후암동,19
432,휘경1동,15
433,휘경2동,17


In [17]:
# bus.to_excel("버스정류장.xlsx")

In [26]:
df = pd.read_excel("최종_버스승객수_버스정류장개수.xlsx")
df

,행정동_ID,행정동,지역구,294일 버스 승객수,하루_버스승객,한달_버스승객,일년_버스승객,하루승객,count_busstop
0,1124066,가락1동,송파구,2098577,7138,214141,2605376,7138,10
1,1124067,가락2동,송파구,1757533,5978,179340,2181971,5978,22
2,1124065,가락본동,송파구,2481986,8442,253264,3081377,8442,25
3,1117070,가리봉동,구로구,1800277,6123,183702,2235038,6123,10
4,1118051,가산동,금천구,8718978,29656,889692,10824582,29656,105
...,...,...,...,...,...,...,...,...,...
419,1103058,효창동,용산구,1013758,3448,103445,1258577,3448,14
420,1103051,후암동,용산구,1538726,5234,157013,1910323,5234,19
421,1106072,휘경1동,동대문구,3015044,10255,307658,3743167,10255,15
422,1106073,휘경2동,동대문구,3071120,10446,313380,3812785,10446,17


In [ ]:
#
fin = pd.read_excel("fin_dataset_zero.xlsx")
fin